In [6]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [7]:
# !pip install sentence_transformers

In [1]:
# !pip install pandas
# !pip install numpy

In [7]:
# !conda info --envs # check which env we are in
# !conda activate HCP_similarity-env # Activate the right environment


In [1]:
# importing tokenized sentence to use for similarity
# load json module
import json
import pandas as pd

with open('Master_indeed_dict.json') as json_file:
    Master_indeed_dict = json.load(json_file)
with open('syllabi_tokenized_dict.json') as json_file:
    syllabi_tokenized_dict = json.load(json_file)
with open('syllabi_NICCS.json') as json_file:
    syllabi_NICCS_dict = json.load(json_file)    
with open('Certs.json') as json_file:
    Certs_dict = json.load(json_file)  

In [4]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('bert-base-nli-mean-tokens')
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [5]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
syllabi_niccs_vecs = {}
for WR in syllabi_NICCS_dict.keys():
    k_vec = model.encode(syllabi_NICCS_dict[WR]['K'])
    s_vec = model.encode(syllabi_NICCS_dict[WR]['S'])
    a_vec = model.encode(syllabi_NICCS_dict[WR]['A'])
    
    syllabi_niccs_vecs[WR] = {'K': k_vec, 
                             'S': s_vec, 
                             'A': a_vec}

C:\Users\15713\anaconda3\envs\HCP-env\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [6]:
syllabi_niccs_vecs['Network Operations Specialist']['K'].shape

(39, 384)

In [7]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
Certs_vecs = {}
for cert in Certs_dict.keys():
    k_vec = model.encode(Certs_dict[cert]['K'])
    s_vec = model.encode(Certs_dict[cert]['S'])
    a_vec = model.encode(Certs_dict[cert]['A'])
    
    Certs_vecs[cert] = {'K': k_vec, 
                        'S': s_vec, 
                        'A': a_vec}

In [8]:
print(Certs_vecs['CISSP (Certified Information Systems Security Professional)']['K'].shape)

(70, 384)


In [ ]:
#####################
## HERE I want to vectorize each sentence while preserving the data structure (instead of a sentence, we will have a vector/list)
Indeed_vecs = {}
for job in Master_indeed_dict.keys():
    k_vec = model.encode(Master_indeed_dict[job]['K'])
    s_vec = model.encode(Master_indeed_dict[job]['S'])
    a_vec = model.encode(Master_indeed_dict[job]['A'])
    
    Master_indeed_dict[job] = {'K': k_vec, 
                               'S': s_vec, 
                               'A': a_vec}

In [ ]:
# Indeed_vecs['Network Operations Specialist']['K'].shape

In [ ]:
# Showing the dataframe
syllabi_vec_df.head(10)

In [116]:
# Showing the dataframe
indeed_vec_df.head(10)

,0
Senior Software Engineer Security and Authentication,"[[-0.5077066, 0.40989426, 1.2277379, -0.270982..."
"2021745 Cyber Security Analysts $215,000.00","[[0.2031807, 0.28579754, 0.748438, 0.21775588,..."
Senior Cyber Security Analyst,"[[-0.9421163, 0.9445789, -0.56157345, 0.447010..."
Cyber Security Analyst,"[[-0.09062232, 0.10185893, 1.6914076, 0.380097..."
Cyber Security Engineer,"[[0.11851995, 0.8113348, 1.4092755, -0.3755570..."
Cyber Security and Networking Apprentice,"[[0.20767999, 0.49081442, 2.3505204, -0.161244..."
Cyber Threat Intelligence Analyst,"[[0.09805263, -0.2946825, 2.3401241, 0.4846215..."
Security Sales Specialist,"[[-0.7816006, 0.8927222, -0.84627825, 0.539904..."
Cyber Security Consultant,"[[-0.26815602, 0.33816063, 1.3826727, 0.144817..."
Cybersecurity Engineer,"[[-0.055971794, 0.6803263, 1.2424763, -0.04021..."


In [ ]:
# Implementing cosine_similarity() funcion from sklearn

In [28]:
from sklearn.metrics.pairwise import cosine_similarity           

In [29]:
# finding similarities b/n all sentences and saving in a dictionary
sim_dict = {}

for K in indeed_vecs.keys():# looping thru keys
    for k in indeed_vecs.keys():# looping thru keys
        if K != k: # Since we don't want to compare the same sentences
            # looping thru the list of sentences within the dict key
            for i,x in enumerate(indeed_vecs[K]): 
                # finding similarity of 1 sentence vs all others
                S = cosine_similarity([indeed_vecs[K][i]],
                                     indeed_vecs[k][:])
                key = f'{K}[{i}]-{k}'
                sim_dict[key] = S

                

In [115]:
index = 0
with open('indeed_similarity.txt', w) as f:
    for K in indeed_vecs.keys():
        try:
            print(f"SENTENCE: \"{indeed_tokenized_dict[K][index]}\" \nFROM \"{K}\"\n\n\t\tIS SIMILAR TO:", file=f)
            for k in indeed_vecs.keys():
                if K != k:
                    print(f'\n[Comparing with sentences in \"{k}\"...]', file=f)
                    key = f'{K}[{index}]-{k}'
                    index+=1
                    for x in sim_dict[key]:
                        for i,y in enumerate(x):
                            if 0.82 < y < 0.99 :
                                print(f"\nFROM: \"{k}\"\nSENTENCE: \"{indeed_tokenized_dict[k][i]}\"\nBY: {y}", file=f)
                    print(f'\n[END OF SENTENCES FOR \"{k}\"]', file=f)
                    print('##########################################################', file=f)
        except:
            print('[End of List Reached]\n\n',file=f)
            print('[Moving to the NEXT sentence to look at...]',file=f)
            index = 0

            

SENTENCE: "We are looking for a highly motivated work-from-home solutions sales professional to drive continued high growth in our successful Cyber security small business sales." 
FROM "Senior Software Engineer Security and Authentication"

		IS SIMILAR TO:

[Comparing with sentences in "None"...]

FROM: "None"
SENTENCE: "Blending cutting-edge technologies, unique skill sets and proven cyber counterintelligence strategies, SpearTip partners with our clients to protect shareholder value, shield corporate reputations and enhance long-term profits."
BY: 0.8261666297912598

FROM: "None"
SENTENCE: "You will be a highly motivated entrepreneurial-minded sales professional who has a real passion for delivering elite technology and services that protect our customers from email-related threats."
BY: 0.8565418720245361

FROM: "None"
SENTENCE: "If you are looking for your next career opportunity and enjoy working in a fast-paced and growing business environment and want to be on a team with a “N


FROM: "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"
SENTENCE: "Assist staff with understanding and adhering to security controls and policies."
BY: 0.8690192699432373

FROM: "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"
SENTENCE: "Develop and maintain governance, risk, and compliance strategy."
BY: 0.8342884182929993

FROM: "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"
SENTENCE: "Information security management, governance, and compliance principles, practices, laws, rules, and regulations."
BY: 0.8489384055137634

[END OF SENTENCES FOR "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"]
##########################################################

[Comparing with sentences in "IT GxP Quality Assurance / CSV Auditor (Remote)"...]

[END OF SENTENCES FOR "IT GxP Quality Assurance / CSV Auditor (Remote)"]
##########################################################

[Comparing with sentences in "Security Architect"...]

FROM: "Security Architect"
SENTENCE: "Bac


[END OF SENTENCES FOR "Security Sales Specialist"]
##########################################################

[Comparing with sentences in "Cyber Security Consultant"...]

[END OF SENTENCES FOR "Cyber Security Consultant"]
##########################################################

[Comparing with sentences in "Cybersecurity Engineer"...]

[END OF SENTENCES FOR "Cybersecurity Engineer"]
##########################################################

[Comparing with sentences in "Information Security Specialist/Auditor"...]

FROM: "Information Security Specialist/Auditor"
SENTENCE: "Yes"
BY: 0.8252248764038086

[END OF SENTENCES FOR "Information Security Specialist/Auditor"]
##########################################################

[Comparing with sentences in "Network Security Engineer"...]

FROM: "Network Security Engineer"
SENTENCE: "Requirements:"
BY: 0.882319450378418

FROM: "Network Security Engineer"
SENTENCE: "REQUIREMENTS"
BY: 0.8598873019218445

[END OF SENTENCES FOR "Network 

[END OF SENTENCES FOR "None"]
##########################################################

[Comparing with sentences in "2021745 Cyber Security Analysts $215,000.00"...]

[END OF SENTENCES FOR "2021745 Cyber Security Analysts $215,000.00"]
##########################################################

[Comparing with sentences in "Senior Cyber Security Analyst"...]

FROM: "Senior Cyber Security Analyst"
SENTENCE: "The company serves as a champion for the growing Creator Economy, enabling individuals to monetize their content, creativity, and influence."
BY: 0.821175217628479

FROM: "Senior Cyber Security Analyst"
SENTENCE: "When we hire at IZEA, it's to add to our strong existing roster of talent and choose the right individual to impact, collaborate, commit, inspire, and grow alongside our tenured teams."
BY: 0.8337217569351196

FROM: "Senior Cyber Security Analyst"
SENTENCE: "We take great pride in creating a positive work environment for all our employees - from our investment in team m


[Comparing with sentences in "Digital Forensics Engineer"...]

[END OF SENTENCES FOR "Digital Forensics Engineer"]
##########################################################

[Comparing with sentences in "Red Team - Penetration Tester"...]

[END OF SENTENCES FOR "Red Team - Penetration Tester"]
##########################################################

[Comparing with sentences in "Cyber Intelligence Analyst"...]

FROM: "Cyber Intelligence Analyst"
SENTENCE: "Ability to establish and maintain effective relationships with other employees and contractors."
BY: 0.8269780874252319

[END OF SENTENCES FOR "Cyber Intelligence Analyst"]
##########################################################

[Comparing with sentences in "Digital Forensics Analyst, Employment Law & Investigations"...]

FROM: "Digital Forensics Analyst, Employment Law & Investigations"
SENTENCE: "‐ Unbeatable benefits include:"
BY: 0.8687325716018677

FROM: "Digital Forensics Analyst, Employment Law & Investigations"
SENTE

...


[END OF SENTENCES FOR "None"]
##########################################################

[Comparing with sentences in "2021745 Cyber Security Analysts $215,000.00"...]

[END OF SENTENCES FOR "2021745 Cyber Security Analysts $215,000.00"]
##########################################################

[Comparing with sentences in "Senior Cyber Security Analyst"...]

[END OF SENTENCES FOR "Senior Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Analyst"...]

FROM: "Cyber Security Analyst"
SENTENCE: "Familiarity with the identification of vulnerable code and a bonus if knowledgeable about possible cyber-attacks such as race conditions, covert channel, replay, and return-oriented attacks."
BY: 0.8200172781944275

FROM: "Cyber Security Analyst"
SENTENCE: "The ideal candidate will have a background in hunting cyber threats, red/purple teaming and large scale data analysis."
BY: 0.8279277682304382

[END OF SENTENCE

[END OF SENTENCES FOR "Sr. Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"...]

FROM: "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"
SENTENCE: "Schedule:"
BY: 0.8272074460983276

[END OF SENTENCES FOR "Cyber Security Analyst Associate SOC Tier 1 ITC#3073"]
##########################################################

[Comparing with sentences in "IT GxP Quality Assurance / CSV Auditor (Remote)"...]

[END OF SENTENCES FOR "IT GxP Quality Assurance / CSV Auditor (Remote)"]
##########################################################

[Comparing with sentences in "Security Architect"...]

[END OF SENTENCES FOR "Security Architect"]
##########################################################

[Comparing with sentences in "Sales/Biz Dev - Cyber Security"...]

[END OF SENTENCES FOR "Sales/Biz Dev - Cyber Security"]
#######################################################


[END OF SENTENCES FOR "Security Sales Specialist"]
##########################################################

[Comparing with sentences in "Cyber Security Consultant"...]

[END OF SENTENCES FOR "Cyber Security Consultant"]
##########################################################

[Comparing with sentences in "Cybersecurity Engineer"...]

[END OF SENTENCES FOR "Cybersecurity Engineer"]
##########################################################

[Comparing with sentences in "Information Security Specialist/Auditor"...]

[END OF SENTENCES FOR "Information Security Specialist/Auditor"]
##########################################################

[Comparing with sentences in "Network Security Engineer"...]

[END OF SENTENCES FOR "Network Security Engineer"]
##########################################################

[Comparing with sentences in "Sr. Cyber Security Analyst"...]

[END OF SENTENCES FOR "Sr. Cyber Security Analyst"]
##########################################################



[END OF SENTENCES FOR "None"]
##########################################################

[Comparing with sentences in "2021745 Cyber Security Analysts $215,000.00"...]

[END OF SENTENCES FOR "2021745 Cyber Security Analysts $215,000.00"]
##########################################################

[Comparing with sentences in "Senior Cyber Security Analyst"...]

[END OF SENTENCES FOR "Senior Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Analyst"...]

[END OF SENTENCES FOR "Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Engineer"...]

[END OF SENTENCES FOR "Cyber Security Engineer"]
##########################################################

[Comparing with sentences in "Cyber Security and Networking Apprentice"...]

FROM: "Cyber Security and Networking Apprentice"
SENTENCE: "We're looking to grow our teams with more peop


[END OF SENTENCES FOR "None"]
##########################################################

[Comparing with sentences in "2021745 Cyber Security Analysts $215,000.00"...]

[END OF SENTENCES FOR "2021745 Cyber Security Analysts $215,000.00"]
##########################################################

[Comparing with sentences in "Senior Cyber Security Analyst"...]

[END OF SENTENCES FOR "Senior Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Analyst"...]

[END OF SENTENCES FOR "Cyber Security Analyst"]
##########################################################

[Comparing with sentences in "Cyber Security Engineer"...]

[END OF SENTENCES FOR "Cyber Security Engineer"]
##########################################################

[Comparing with sentences in "Cyber Security and Networking Apprentice"...]

[END OF SENTENCES FOR "Cyber Security and Networking Apprentice"]
########################################